# Cross means

The compare means t-test is used to compare the mean of a variable in one group to the mean of the same variable in one, or more, other groups. The null hypothesis for the difference between the groups in the population is set to zero. We test this hypothesis using sample data.

In [1]:
import matplotlib as mpl
import pyrsm as rsm

# increase plot resolution
mpl.rcParams["figure.dpi"] = 150

ModuleNotFoundError: No module named 'pyrsm'

In [ ]:
rsm.load_data(pkg="basics", name="salary", dct=globals())

In [ ]:
rsm.describe(salary)

In [ ]:
salary

In [ ]:
cm = rsm.compare_means(salary, "rank", "salary", [("AsstProf", "AssocProf"), ("AsstProf", "Prof"), ("AssocProf", "Prof")], "less", 0.95)
cm.calculate()
cm.summary(dec=3)